In [97]:
import numpy as np
from numpy import count_nonzero
import pyreadstat
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn import scatterplot as scatter
from scipy.sparse import csr_matrix
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA

from sklearn.semi_supervised import LabelSpreading
from sklearn.semi_supervised import LabelPropagation

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report


from sklearn import metrics
#from sklearn.model_selection import cross_val_score
#import sklearn.metrics.scorer as scorer


#reading data from file

#df,meta=pyreadstat.read_sav("C:/Users/APPLE/Desktop/Folders/sana thesis/SHAHNAZ-meysam-tekvando.sav")
df=pd.read_excel("C:/Users/APPLE/Desktop/Folders/sana thesis/SHAHNAZ-meysam-tekvandolabelmodified.xls")

#preprocessing operations such as converting Nan values to zero,
#converting fields without value to zero,
#removing ID,fisrtname, and familyname columns from dataset

features_name=df.columns
data=np.array(df)
df=pd.DataFrame(data=data,columns=features_name)
df=df.drop(['firstname','familyname','year'],axis=1)

# keep only women samples
df1=df
for l in range(2559):
    if (df1.iloc[l,0]==1.0):
        df=df.drop([l])    

true_labels=df.iloc[:,-1]
df=df.drop(['label','gender'],axis=1)
features_name=df.columns

true_labels=true_labels.fillna(0)
for i in range(len(true_labels)):
    if (true_labels.iloc[i]==0):
        true_labels.iloc[i]=-1

In [98]:
df=df.drop(['Concani','burus','wingetAP','WingtePP','ricaveri','oneMayl','pareshTool','ghodratPanjeh','barfix','jaheshJanebi',
            'FlexibleShoulder','RTshenidari','tawanBiHavazi','biHAVA1609','Do1600','yekMayl',
            'metr540','taadol'],axis=1)
features_name=df.columns

In [99]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer
from sklearn.impute import MissingIndicator
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor

data=np.array(df)
data=data.astype('float')
data[data==0.0]=np.nan
df=pd.DataFrame(data=data,columns=features_name)

strategy=('mean','median')
simpleimputer=SimpleImputer(missing_values=np.nan, strategy=strategy[0])
data1=simpleimputer.fit_transform(df)
#df1=pd.DataFrame(data=data1,columns=features_name)
#simpleimputer=SimpleImputer(missing_values=0.0, strategy=strategy[0])
#data1=simpleimputer.fit_transform(df1)


knnimputer=KNNImputer(missing_values=np.nan)
data2=knnimputer.fit_transform(df)
#df2=pd.DataFrame(data=data2,columns=features_name)
#knnimputer=KNNImputer(missing_values=0.0)
#data2=knnimputer.fit_transform(df2)

estimators=[BayesianRidge(),
            DecisionTreeRegressor(max_features='sqrt',random_state=0),
            ExtraTreesRegressor(n_estimators=10, random_state=0),
            KNeighborsRegressor(n_neighbors=15)
           ]

iterativeimputer=IterativeImputer(missing_values=np.nan, random_state=0, estimator=estimators[3], n_nearest_features=5)
data3=iterativeimputer.fit_transform(df)
#df3=pd.DataFrame(data=data3,columns=features_name)
#iterativeimputer=IterativeImputer(missing_values=0.0, random_state=0, estimator=estimators[0], n_nearest_features=5)
#data3=iterativeimputer.fit_transform(df3)

C:\Users\APPLE\AppData\Roaming\Python\Python37\site-packages\sklearn\impute\_iterative.py:638: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [100]:
#performing standardScaler as preprocessing

scaler=StandardScaler(with_mean=False)
data_scaled=scaler.fit_transform(data2)
df=pd.DataFrame(data=data_scaled,columns=features_name)

In [101]:
#keep 60% of labeled data for test
true_labels=true_labels.astype(float)
true_labels.reset_index(inplace=True, drop=True)

df['label']=true_labels
df3=df
df4=df

# making df3 as a labeled set
for l in range(len(df)):
    if(df.iloc[l,-1]==-1):
        df3=df3.drop([l])
        
df3_labels=df3.iloc[:,-1]
df5=df3
#df3=df3.drop(['label'],axis=1)
X_train_labeled,X_test=train_test_split(df3,test_size=0.60,random_state=42)

#making df4 as an unlabeled set
for l in range(len(df)):
    for i in range(len(df5)):
        if (df.iloc[l,:].equals(df5.iloc[i,:])):
            df4=df4.drop([l])
            break

X_train=df4
X_train_unlabeled=X_train
X_train=X_train.append(X_train_labeled)

Y_train_labeled=X_train_labeled.iloc[:,-1]
X_train_labeled=X_train_labeled.drop(['label'],axis=1)
Y_train_labeled=Y_train_labeled.to_frame()
Y_train_labeled=Y_train_labeled.astype(float)
Y_train_labeled=Y_train_labeled.reset_index(drop=True)
X_train_labeled=X_train_labeled.astype(float)
X_train_labeled=X_train_labeled.reset_index( drop=True)

X_train_unlabeled=X_train_unlabeled.drop(['label'],axis=1)
X_train_unlabeled=X_train_unlabeled.astype(float)
X_train_unlabeled=X_train_unlabeled.reset_index( drop=True)



Y_train=X_train.iloc[:,-1]
Y_test=X_test.iloc[:,-1]
X_train=X_train.drop(['label'],axis=1)
X_test=X_test.drop(['label'],axis=1)
Y_train=Y_train.astype(float)
Y_train=Y_train.reset_index(drop=True)
Y_test=Y_test.astype(float)
Y_test=Y_test.reset_index( drop=True)
X_train=X_train.astype(float)
X_test=X_test.astype(float)
X_train=X_train.reset_index( drop=True)
X_test=X_test.reset_index( drop=True)
Y_train=Y_train.to_frame()
Y_test=Y_test.to_frame()

In [89]:
data=np.array(X_train)
label=np.array(Y_train)

#feature extraction algorithms

#svd = TruncatedSVD(n_components=2, algorithm='randomized')
#data_extracted=svd.fit_transform(data)

#nmf=NMF(n_components=2, init='random', random_state=0)
#data_extracted=nmf.fit_transform(data)

pca=PCA(n_components=2)
data_extracted=pca.fit_transform(data)

#kpca=KernelPCA(n_components=2,kernel='poly')
#data_extracted=kpca.fit_transform(data)

#lda=LatentDirichletAllocation(n_components=2, random_state=0)
#data_extracted=lda.fit_transform(data)

#kpca=KernelPCA(n_components=2,kernel='cosine')
#data_extracted=kpca.fit_transform(data_extracted)

#fastICA=FastICA(n_components=2, random_state=0)
#data_extracted=fastICA.fit_transform(data)

#fa=FactorAnalysis(n_components=2, random_state=0)
#data_extracted=fa.fit_transform(data)

#dl=DictionaryLearning(n_components=2, random_state=0, fit_algorithm='cd',transform_algorithm='lasso_cd')
#data_extracted=dl.fit_transform(data)
#------------------------------------------------------------------------------------------
#Kfold splitting 
#kfold=KFold(n_splits=5, random_state=1)
#k=0
#for train, test in kfold.split(data_extracted):
#    k=k+1
#    label_propagation_model=LabelPropagation()
#    label_propagation_model.fit(data_extracted[train],label[train])
#    test_indices_truelabeled_list=[]
#    test_true_labels=[]
#    test_data_extracted=data_extracted[test]
#    for l in range(len(data_extracted[test])):
#        if (label[train][l]!=-1):
           #labeled_data_extracted.iloc[i,:]=test_data_extrated.iloc[l,:]
#            test_indices_truelabeled_list.append(l)
#            test_true_labels.append(label[train][l])
            
            
#    predicted_labels=label_propagation_model.transduction_[test_indices_truelabeled_list]
#    cm=confusion_matrix(test_true_labels,predicted_labels,labels=label_propagation_model.classes_)
#---------------------------------------------------------------------------------------
#cl=LabelPropagation(kernel="rbf",gamma=15,n_neighbors=4)
cl=LabelSpreading(kernel="knn",n_neighbors=8)
cvresult1=crossvalidationSSL(data_extracted,label,cl,"test",3)
#cl=crossvalidationSSL(data_extracted,label,svc,"train",3)
#cvrsult2=crossvaidationSSL(X,y,classifier,measures,leaveout,k,repeats,n_labeled)

C:\Users\APPLE\AppData\Roaming\Python\Python37\site-packages\sklearn\semi_supervised\_label_propagation.py:288: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning


              precision    recall  f1-score   support

         1.0       0.46      0.50      0.48        12
         2.0       0.33      0.33      0.33         6
         3.0       0.25      0.20      0.22         5

    accuracy                           0.39        23
   macro avg       0.35      0.34      0.35        23
weighted avg       0.38      0.39      0.39        23

Confusion matrix
[[6 3 3]
 [4 2 0]
 [3 1 1]]


C:\Users\APPLE\AppData\Roaming\Python\Python37\site-packages\sklearn\semi_supervised\_label_propagation.py:288: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning


              precision    recall  f1-score   support

         1.0       0.53      0.89      0.67         9
         2.0       1.00      0.10      0.18        10
         3.0       0.29      0.50      0.36         4

    accuracy                           0.48        23
   macro avg       0.61      0.50      0.40        23
weighted avg       0.69      0.48      0.40        23

Confusion matrix
[[8 0 1]
 [5 1 4]
 [2 0 2]]
              precision    recall  f1-score   support

         1.0       0.37      0.88      0.52         8
         2.0       0.00      0.00      0.00         3
         3.0       1.00      0.08      0.15        12

    accuracy                           0.35        23
   macro avg       0.46      0.32      0.22        23
weighted avg       0.65      0.35      0.26        23

Confusion matrix
[[7 1 0]
 [3 0 0]
 [9 2 1]]


C:\Users\APPLE\AppData\Roaming\Python\Python37\site-packages\sklearn\semi_supervised\_label_propagation.py:288: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning


In [146]:
from sklearn.decomposition import KernelPCA
data_test=np.array(X_test)
y_true=np.array(Y_test)
Y_train=np.array(Y_train)
data_train=np.array(X_train)
#data_train=np.array(X_train_labeled)
whole_data=np.array(df)

#pca=PCA(n_components=2)
#train_data_extracted=pca.fit_transform(data_train)
#test_data_extracted=pca.fit_transform(data_test)
#whole_data_extracted=pca.fit_transform(whole_data)

svd = TruncatedSVD(n_components=2, algorithm='randomized')
train_data_extracted=svd.fit_transform(data_train)
test_data_extracted=svd.fit_transform(data_test)
whole_data_extracted=svd.fit_transform(whole_data)

#nmf=NMF(n_components=2, init='random', random_state=0)
#train_data_extracted=nmf.fit_transform(data_train)
#test_data_extracted=nmf.fit_transform(data_test)
#whole_data_extracted=nmf.fit_transform(whole_data)

#lda=LatentDirichletAllocation(n_components=2, random_state=0)
#train_data_extracted=lda.fit_transform(data_train)
#test_data_extracted=lda.fit_transform(data_test)
#whole_data_extracted=lda.fit_transform(whole_data)

#kpca=KernelPCA(n_components=2,kernel='cosine')
#train_data_extracted=kpca.fit_transform(data_train)
#test_data_extracted=kpca.fit_transform(data_test)
#whole_data_extracted=kpca.fit_transform(whole_data)
    
#fastICA=FastICA(n_components=2, random_state=0)
#train_data_extracted=fastICA.fit_transform(data_train)
#test_data_extracted=fastICA.fit_transform(data_test)
#whole_data_extracted=fastICA.fit_transform(whole_data)

cl=LabelSpreading(kernel="knn",gamma=20,n_neighbors=7)
#cl=LabelPropagation(kernel="rbf",gamma=15,n_neighbors=6)

cl.fit(train_data_extracted,Y_train)
y_predicted=cl.predict(test_data_extracted)
cm=confusion_matrix(y_true, y_predicted, labels=cl.classes_)


print(classification_report(y_true, y_predicted))
print("confusion_matrix")
print(cm)


              precision    recall  f1-score   support

         1.0       0.44      0.63      0.52        38
         2.0       0.55      0.24      0.33        46
         3.0       0.29      0.43      0.35        21

    accuracy                           0.42       105
   macro avg       0.43      0.43      0.40       105
weighted avg       0.46      0.42      0.40       105

confusion_matrix
[[24  5  9]
 [22 11 13]
 [ 8  4  9]]


C:\Users\APPLE\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\APPLE\AppData\Roaming\Python\Python37\site-packages\sklearn\semi_supervised\_label_propagation.py:293: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer


In [ ]:
#plot the result
data_test=np.array(X_test)
#y_true=np.array(Y_test)
#Y_test=Y_test.iloc[:,-1]
data_train=np.array(X_train)
#data_train=np.array(X_train_labeled)
whole_data=np.array(df)

pca=PCA(n_components=2)
train_data_extracted=pca.fit_transform(data_train)
test_data_extracted=pca.fit_transform(data_test)
whole_data_extracted=pca.fit_transform(whole_data)

#svd = TruncatedSVD(n_components=2, algorithm='randomized')
#train_data_extracted=svd.fit_transform(data_train)
#test_data_extracted=svd.fit_transform(data_test)
#whole_data_extracted=svd.fit_transform(whole_data)

#fastICA=FastICA(n_components=2, random_state=0)
#train_data_extracted=fastICA.fit_transform(data_train)
#test_data_extracted=fastICA.fit_transform(data_test)
#whole_data_extracted=fastICA.fit_transform(whole_data)


#labelspreading=LabelSpreading(kernel='knn',gamma=8, n_neighbors=7)
#learntransductive(train_data_extracted,Y_train,test_data_extracted,Y_test,whole_data_extracted,true_labels,
#                  labelspreading,"LabelSpreading","test data","knn,n_neighbors=7,SVD")
labelpropagation=LabelPropagation(kernel='rbf',gamma=15)

learntransductive(train_data_extracted,Y_train,test_data_extracted,Y_test,whole_data_extracted,true_labels,labelpropagation,"LabelPropagation","test data","rbf,gamma=15,PCA")
    

In [95]:
def learntransductive(data_train_extracted,Y_train,data_test_extracted,y_true,whole_data_extracted,true_labels,
                      classifier,methodname,kindofdata,kernel):

    
    classifier.fit(whole_data_extracted,true_labels)
    kindofdata="whole data"
    output_labels=classifier.transduction_
    plottransductive(whole_data_extracted,true_labels,output_labels,kindofdata,methodname,kernel)
    kindofdata="test data"
    classifier.fit(data_train_extracted,Y_train)
    y_predicted=classifier.predict(data_test_extracted)
    plottransductive(data_test_extracted,y_true,y_predicted,kindofdata,methodname,kernel)
    
    
def plottransductive(data,true_labels,output_labels,kindofdata,methodname,kernel):

    #plot the result
    
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.scatter(data[true_labels==1,0],data[true_labels==1,1],color='navy',marker='s', lw=0, label="gold medal", s=10)
    plt.scatter(data[true_labels==2,0],data[true_labels==2,1],color='c',marker='s', lw=0, label="silver medal", s=10)
    plt.scatter(data[true_labels==3,0],data[true_labels==3,1],color='r',marker='s', lw=0, label="bronze medal", s=10)
    plt.scatter(data[true_labels==-1,0],data[true_labels==-1,1],color='darkorange',marker='.', lw=0, label="unknown labels", s=10)
    plt.legend(scatterpoints=1, shadow=False, loc='upper right')
    plt.title("Raw data (3 classes=gold,silvar and bronze)")

    plt.subplot(1,2,2)
    output_label_array = np.asarray(output_labels)
    gold_numbers = np.where(output_label_array == 1)[0]
    siver_numbers = np.where(output_label_array == 2)[0]
    bronze_numbers = np.where(output_label_array == 3)[0]

    plt.scatter(data[gold_numbers, 0], data[gold_numbers, 1], color='navy',
            marker='s', lw=0, s=10, label="gold medal")
    plt.scatter(data[siver_numbers, 0], data[siver_numbers, 1], color='c',
            marker='s', lw=0, s=10, label="silver medal")
    plt.scatter(data[bronze_numbers, 0], data[bronze_numbers, 1], color='r',
            marker='s', lw=0, s=10, label="bronze medal")
    plt.legend(scatterpoints=1, shadow=False, loc='upper right')
    plt.title("Labels of "+kindofdata+" learned with "+methodname+" (kernel="+kernel+")")

    plt.subplots_adjust(left=0.07, bottom=0.07, right=0.93, top=0.92)
    plt.show()


In [18]:
def crossvalidationSSL(X,y,classifier,leaveout,k):
    df=pd.DataFrame(X)
    df['label']=y
    df2=df
    df3=df
    #making df2 as a labeled DataFrame:
    for l in range(len(df)):
        if(df.iloc[l,-1]==-1):
            df2=df2.drop([l])
    #making df3 as an unLabeled DataFrame:
    for l in range(len(df)):
        for i in range(len(df2)):
            if (df.iloc[l,:].equals(df2.iloc[i,:])):
                df3=df3.drop([l])
    if (leaveout=="test"):
        #making validation and train sets:
        #using labeled sets just for validation and 
        #using whole unlabeled set plus each fold for training in each iteration.
        validationDataFrameList=np.array_split(df2,k)
        validationDataFrameList2=validationDataFrameList
        trainDataFrameList=[None]*k
        for l in range(k):
            trainDataFrameList[l]=df3
            
        for l in range(k):
            removed=validationDataFrameList.pop()
            trainDataFrameList[l]=trainDataFrameList[l].append(validationDataFrameList)
            validationDataFrameList.insert(0,removed)
        validationDataFrameList2.reverse()
        for l in range(k):
            cl=classifier
            y=trainDataFrameList[l].iloc[:,-1]
            X=np.array(trainDataFrameList[l].drop(['label'],axis=1))
            cl.fit(X,y)
            y_true=validationDataFrameList2[l].iloc[:,-1]
            removedValidationDataFrame=validationDataFrameList2[l].drop(['label'],axis=1)
            y_predicted=cl.predict(removedValidationDataFrame)
            
            cm = confusion_matrix(y_true, y_predicted, labels=cl.classes_)
            
            print(classification_report(y_true, y_predicted))

            print("Confusion matrix")
            print(cm)
    elif (leaveout=='train'):
        #making train and validation sets:
        #using k-1 labeled sets for training and one set for validation in each iteration
        foldDataFrameList=np.array_split(df2,k)
        foldDataFrameList2=foldDataFrameList
        trainDataFrameList=[None]*k
        for l in range(k):
            removed=foldDataFrameList.pop()
            trainDataFrameList[l]=foldDataFrameList
            foldDataFrameList.insert(0,removed)
        foldDataFrameList2.reverse()
        for i in range(k):
            cl=classifier
            temp=trainDataFrameList[i]
            trainDataFrame=pd.DataFrame()
            for j in range(k):
                trainDataFrame=trainDataFrame.append(temp[j],ignore_index=True)
            X=np.array(trainDataFrame.drop(['label'],axis=1))
            y=trainDataFrame.iloc[:,-1]
            cl.fit(X,y)
            y_true=foldDataFrameList2[i].iloc[:,-1]
            validationDataFrame=foldDataFrameList2[i].drop(['label'],axis=1)
            y_predicted=cl.predict(validationDataFrame)
            
            cm = confusion_matrix(y_true, y_predicted, labels=cl.classes_)
            
            print(classification_report(y_true, y_predicted))

            print("Confusion matrix")
            print(cm)
    return cl
            
#validation_indices_truelabeled=[]
#validation_true_labels=[]
#    test_data_extracted=data_extracted[test]
#    for l in range(len(data_extracted[test])):
#        if (label[train][l]!=-1):
           #labeled_data_extracted.iloc[i,:]=test_data_extrated.iloc[l,:]
#            test_indices_truelabeled_list.append(l)
#            test_true_labels.append(label[train][l])